<a href="https://colab.research.google.com/github/mgerlach/advent_of_code/blob/main/2025/aoc2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>